In [ ]:
import equinox as eqx
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import importlib
import numpy as np
from jax import grad, jit, vmap
from jax.experimental.ode import odeint

from typing import TYPE_CHECKING, Callable, Union, Optional

from VariablesClass import VariablesClass
from StructureClass import  StructureClass
from StateClass import StateClass
from EquilibriumClass import EquilibriumClass

import plot_funcs, colors, dynamics, helpers_builders

In [ ]:
## Params

H, S = 5, 1  # # Hinges, # Shims per hinge

# --- parameters / variables ---
k_soft  = jnp.ones((H, S), dtype=jnp.float32) * 1.0  # (Hinges, Shims) spring stiffnesses in soft direction, per shim
k_stiff = jnp.ones((H, S), dtype=jnp.float32) * 4.0  # (Hinges, Shims) spring stiffnesses in stiff direction, per shim
thetas_ss = jnp.full((H, S), -1/8, dtype=jnp.float32)  # (Hinges, Shims) rest angles per shim
buckle  = jnp.ones((H, S), dtype=jnp.int32)  # initial cubkle state, per shim

T = 2  # total training set time (not time to reach equilibrium during every step)

In [ ]:
import StructureClass
importlib.reload(StructureClass)
from StructureClass import StructureClass

# --- build geometry (all topology stays in StructureClass) ---
Strctr = StructureClass(hinges=H, shims=S, L=1)  # your StructureClass from earlier
# Strctr.build_dataset

In [ ]:
import VariablesClass
importlib.reload(VariablesClass)
from VariablesClass import VariablesClass

# --- Initiate variables ---
Variabs = VariablesClass(Strctr,
                         T,
                         k_soft=k_soft,
                         k_stiff=k_stiff,
                         thetas_ss=thetas_ss,           # rest/target angles
                         stretch_scale=10.0,              # k_stretch = 50 * max(k_stiff)
                         )

In [ ]:
import StateClass
importlib.reload(StateClass)
from StateClass import StateClass

# --- state (straight chain, unit spacing => rest lengths = 1) ---
State = StateClass(Variabs, Strctr)  # buckle defaults to +1

In [ ]:
import EquilibriumClass
importlib.reload(EquilibriumClass)
from EquilibriumClass import EquilibriumClass

# --- state (straight chain, unit spacing => rest lengths = 1) ---
Eq = EquilibriumClass(Strctr, State.buckle, State.pos_arr)

In [ ]:
importlib.reload(dynamics)

State._save_data(0, Strctr, State.pos_arr, State.buckle, compute_thetas_if_missing=True)

for t in range(Variabs.T):
    print('t=', t)
    tip_loc = np.array([5, 0.01])
    final_pos, pos_in_t, vel_in_t, potential_force_evolution = Eq.calculate_state(Variabs, Strctr, tip_loc=tip_loc)
    State._save_data(t, Strctr, final_pos, State.buckle, compute_thetas_if_missing=True)
    plot_funcs.plot_arm(final_pos, Strctr.L)

In [ ]:
Strctr.all_edge_lengths(final_pos)

In [ ]:
from IPython.display import HTML
importlib.reload(plot_funcs)

plot_funcs.plot_arm(final_pos, Strctr.L)

fig, anim = plot_funcs.animate_arm(pos_in_t, Strctr.L, interval_ms=20, save_path=None, fps=1, stride=12)

# Show inline animation (uses JavaScript/HTML5)
HTML(anim.to_jshtml())